Library Imports

In [58]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt


In [59]:
# Define path to the data directory
data_dir = Path('data/binary_split')

data_dir

WindowsPath('data/binary_split')

In [60]:
train_dir = data_dir / 'train'
print(train_dir)
test_dir = data_dir / 'test'

tumor_dir = train_dir / 'tumor'
no_tumor_dir = train_dir / 'no_tumor'
print(tumor_dir)

data\binary_split\train
data\binary_split\train\tumor


In [62]:
from pathlib import Path
import glob

import os

data_dir = Path("data/binary_split")
tumor_dir = data_dir / "train" / "tumor"

print("CWD:", os.getcwd())
print("Path:", tumor_dir.resolve())
print("Dir exists:", tumor_dir.exists())
print("Contents:", list(tumor_dir.iterdir())[:5])


data_dir = Path("data/binary_split")
train_dir = data_dir / "train"
tumor_dir = train_dir / "tumor"
no_tumor_dir = train_dir / "no_tumor"

# Method 1 (glob)
new = glob.glob(str(tumor_dir / "*.jpg"))
print(f"Found {len(new)} image files in the tumor directory (glob).")

# Method 2 (Pathlib)
tumor_cases = list(tumor_dir.glob("*.jpg"))
no_tumor_cases = list(no_tumor_dir.glob("*.jpg"))

print(f"Tumor Cases: {len(tumor_cases)}")
print(f"No Tumor Cases: {len(no_tumor_cases)}")

# See some file names
for case in tumor_cases[:5]:
    print(case.name)


CWD: d:\ML_Projects\brainTumorDetection\notebooks
Path: D:\ML_Projects\brainTumorDetection\notebooks\data\binary_split\train\tumor
Dir exists: False


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data\\binary_split\\train\\tumor'